# RecListsTest
Notebook to store the recommendation lists of the trained models, as well as the test set.

In [1]:
from recbole.quick_start import load_data_and_model
import pandas as pd
from tqdm import tqdm
from recbole.utils.case_study import full_sort_topk
import os
from recsyslearn.accuracy.metrics import NDCG
from recsyslearn.dataset.utils import find_relevant_items
import numpy as np

In [2]:
from recsyslearn.fairness.metrics import FairnessMetric

In [3]:
from recsyslearn.fairness.utils import eff_matrix
from recsyslearn.utils import check_columns_exist

In [7]:
def rec_lists_over_folder(
    folder,
    k=50,
):
    """
    Iterates over the recbole models in the folder passed as argument, 
    computes the metrics and saves the corresponding reclists as .tsv
    
    :param str folder: the folder over which to iterate
    """

    
    for model_file in tqdm(os.listdir(folder)):
        model_name = model_file[:-4]
        model_path = os.path.join(folder, model_file)
        if model_file[-5].isdigit():
            print(model_file)
            # Get the train, test, and topk dataframes from the recbole file
            top_k_df = convert_recbole_tensor_to_recsyslearn_dataframe(model_file=model_path, k=k)
            top_k_df.to_csv(folder + model_name + '.tsv', sep='\t', index=None)

In [5]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [6]:
def convert_recbole_tensor_to_recsyslearn_dataframe(
        model_file,
        k=10,
        device='cpu',
        chunk_size=1000,
        which_set='test',
):
    """
    Function that converts the tensor of shape
    (n_users, k) storing the top k recommendations
    for each user to a dataframe with columns ['user', 'item', 'rank']
    as required by RecSysLearn.

    The steps are
     - load the stored model, as well as the config, and the dataset
     - convert the recbole internal user ids to the tokens of the original dataset
     - get the top k's as a pytorch tensor
     - convert the rows of the tensors to lists of tokens
     - convert everything to a dataframe


    Args:
        model_file: path to the model saved by recbole
        k: the length of the list to be returned
    Returns:
        top_k_df: DataFrame with columns (‘user’, ‘item’, ‘rank’), where each user
        appears in k rows (one row for each item of the k-length list).

    """

    config, model, dataset, _, val_data, test_data = load_data_and_model(
        model_file=model_file,
    )

    if which_set=='val':
        test_data = val_data
    uid_series = range(1, dataset.user_num)
    # print(dataset.user_num, test_data.dataset.user_num)
    top_k_df = pd.DataFrame(columns=['user','item','rank'])
    
    for user_chunk in tqdm(chunker(uid_series, chunk_size)):
        topk_score, topk_iid_list = full_sort_topk(
            user_chunk, model, test_data, k=k, device=device,
        )
        
        u_token_series = dataset.id2token(dataset.uid_field, user_chunk)
        external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
        
        chunk_top_k_df = pd.DataFrame()
        chunk_top_k_df.insert(0, 'user', u_token_series)
        chunk_top_k_df.insert(1, 'item', list(external_item_list))
        chunk_top_k_df.insert(2, 'rank', [range(1, k+1)] * len(chunk_top_k_df))
        
        
        top_k_df = pd.concat([top_k_df, chunk_top_k_df])

    top_k_df = top_k_df.explode(['item', 'rank'])
    return top_k_df

def convert_recbole_dataset_to_recsyslearn_dataframe(
        model_file,
        which_set='test',
        return_rank=False,
        chunk_size=10000,
):
    """
    TODO
     if the dataset contains ratings, exclude interactions below some threshold.

    Function that converts the tensor of shape
    (n_users, ***) and the tensor of shape
    (n_users, ***) storing the internal item id's
    interacted with by the user with the id in that row,
    as well as the rating,
    to a dataframe with columns ['user', 'item', 'rank']
    as required by RecSysLearn.

    REMARK!! The row with index 0 is a dummy padding user
    with no user item interaction entries.

    The steps are
     - load the stored model, as well as the config, and the dataset
     - convert the recbole internal user ids to the tokens of the original dataset
     - get the top k's as a pytorch tensor
     - convert the rows of the tensors to lists of tokens
     - convert everything to a dataframe


    Args:
        model_file: path to the model saved by recbole
        which_set: string marking which dataset has to be converted (full, train, val, or test).
                   default is train.
    Returns:
        history_df: DataFrame with columns (‘user’, ‘item’), with a row for
                    each element of the user-item interaction history.

    """

    _, _, dataset, train_data, valid_data, test_data = load_data_and_model(
        model_file,
    )
    

    if which_set == 'full':
        dataset = dataset
    elif which_set == 'train':
        dataset = train_data.dataset
    elif which_set == 'val':
        dataset = valid_data.dataset
    elif which_set == 'test':
        dataset = test_data.dataset
        
    uid_series = range(1, dataset.user_num)

    i = 0
    for user_chunk in tqdm(chunker(uid_series, chunk_size)):   
        i += 1
        u_token_series = dataset.id2token(dataset.uid_field, user_chunk)
        
        interacted_item_ids = dataset.history_item_matrix()[0][user_chunk]
        external_item_list = dataset.id2token(dataset.iid_field, interacted_item_ids.cpu())

        interacted_ratings = dataset.history_item_matrix()[1][user_chunk].numpy()

        chunk_history_df = pd.DataFrame()
        chunk_history_df.insert(0, 'user', u_token_series)
        chunk_history_df.insert(1, 'item', list(external_item_list))
        chunk_history_df.insert(2, 'rank', list(interacted_ratings))

        chunk_history_df = chunk_history_df.explode(['item', 'rank'])
        # ====
        # chunk_history_df = chunk_history_df[['user', 'item']]
        # ===
        chunk_history_df.to_csv(which_set + '/chunk' + str(int(i)) + '.tsv', sep='\t', index=None)

    chunks = []
    for chunk_file in os.listdir(which_set + '/'):
        # print(chunk_file)
        if chunk_file[:5]=='chunk':
            
            chunk_path = os.path.join(which_set + '/', chunk_file)
            chunk = pd.read_csv(chunk_path, sep='\t')
            chunk['rank'] = chunk['rank'].astype(int)
            # print(chunk.size)
            chunk = chunk[chunk['item'] != '[PAD]']
            # print(chunk.info())
            chunks += [chunk]
            os.remove(chunk_path)

    # print("here")
    history_df = pd.concat(chunks)
    #print(history_df.head())
#    history_df = history_df.set_index('user')
    #bad_df = history_df.item.isin(['[PAD]'])
    #print("here")
    #history_df = history_df[~bad_df]
    # history_df = history_df[history_df['item'] != '[PAD]']
    if return_rank:
        return history_df#[['user', 'item', 'rank']]
    else:
        return history_df[['user', 'item']]

In [ ]:
harmful_list = [0.00, 0.01, 0.05, 0.10]

# DATASET_LIST = ['ml-100k']
DATASET_LIST = ['ml-100k']
#DATASET_LIST = ['lastfm']
BASE_FOLDER = '/home/marta/jku/activity_fair/'
MODEL_LIST = ['MultiVAE']#, 'ItemKNN', 'BPR', ]
model_dict = {
    'BPR': 'bpr',
    'ItemKNN': 'iknn',
    'MultiVAE': 'vae_hyperparams',
}


for MODEL in MODEL_LIST:
    MODEL_SUBFOLDER = model_dict[MODEL]
    for DATASET in DATASET_LIST:
        for harmful in harmful_list:
            DS_STRING = f'{DATASET}_harm{str(int(100*harmful)).zfill(2)}'
            
            MODEL_FOLDER = f'{BASE_FOLDER}saved/{MODEL_SUBFOLDER}/{DS_STRING}/'
            
            
            rec_lists_over_folder(
                folder=MODEL_FOLDER,
            )

 /home/marta/jku/activity_fair/saved/ml-100k_harm01
show_progress = True
save_dataset = True
dataset_save_path = /home/marta/jku/activity_fair/recbole_dataset/
save_dataloaders = True
dataloaders_save_path = /home/marta/jku/activity_fair/recbole_dataloader/
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 1024
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.6, 0.2, 0.2]}, 'fold': 0, 'group_by': 'none', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['NDCG']
topk = [10]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATIN

phases: 3	 datasets: 3


16 Feb 14:47    INFO  Saving split dataloaders into: [/home/marta/jku/activity_fair/saved/ml-100k_harm01/ml-100k_harm01-for-MultiVAE-dataloader.pth]
16 Feb 14:47    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
16 Feb 14:47    INFO  [Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [0.6, 0.2, 0.2]}, 'fold': 0, 'group_by': 'none', 'order': 'RO', 'mode': 'full'}]

0it [00:00, ?it/s]

Error for i =  633


1it [00:00, 13.01it/s]
 19%|████████▎                                  | 10/52 [00:08<00:32,  1.30it/s]

...

Error for i =  31
Error for i =  282
Error for i =  335



1it [00:00,  3.49it/s]
 56%|███████████████████████▉                   | 29/52 [00:30<00:26,  1.16s/it]

MultiVAE-Feb-15-2023_18-19-05.pth


16 Feb 14:49    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /home/marta/jku/activity_fair/datasets/filtered_datasets/ml-100k_harm10
checkpoint_dir = /home/marta/jku/activity_fair/saved/ml-100k_harm10
show_progress = True
save_dataset = True
dataset_save_path = /home/marta/jku/activity_fair/recbole_dataset/
save_dataloaders = True
dataloaders_save_path = /home/marta/jku/activity_fair/recbole_dataloader/
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 1024
learner = adam
learning_rate = 0.01
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.6, 0.2, 0.2]}, 'fold': 0, 'group_by': 'none', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['NDCG']
topk 

phases: 3	 datasets: 3

In [ ]:
harmful_list = [0.00, 0.01, 0.05, 0.10]

# DATASET_LIST = ['ml-100k']
DATASET_LIST = ['ml-100k']
#DATASET_LIST = ['lastfm']
BASE_FOLDER = '/home/marta/jku/activity_fair/'
MODEL_LIST = ['MultiVAE']#, 'ItemKNN', 'BPR', ]
model_dict = {
    'BPR': 'bpr',
    'ItemKNN': 'iknn',
    'MultiVAE': 'vae',
}


for MODEL in MODEL_LIST:
    MODEL_SUBFOLDER = model_dict[MODEL]
    for DATASET in DATASET_LIST:
        for harmful in harmful_list:
            DS_STRING = f'{DATASET}_harm{str(int(100*harmful)).zfill(2)}'

            MODEL_FOLDER = f'{BASE_FOLDER}saved/{MODEL_SUBFOLDER}/{DS_STRING}/'
            MODEL_FILE = MODEL_FOLDER + 'best_model.pth'
            TEST_DATA_FILE = f'{BASE_FOLDER}/test/{DS_STRING}.tsv'
            TEST_DATASET_RANK = convert_recbole_dataset_to_recsyslearn_dataframe(
                model_file=MODEL_FILE,
                which_set='test', 
                return_rank=True
            )
            TEST_DATASET_RANK.to_csv(TEST_DATA_FILE, sep='\t', index=None)
            rec_lists_over_folder(MODEL_FOLDER)